In [ ]:
import json
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
model_name = "unsloth/Qwen2.5-1.5B-Instruct"
with open("/home/mshahidul/project1/all_tran_data/filtered_alignment_result.json") as f:
    data = json.load(f)

# Convert to the required format
converted_dataset = []

for entry in data:
    conversation={}
    conversation['conversations'] = [
        {"from": "human", "value": f"Translate the sentence from English to Spanish: {entry['english']}"},  
        {"from": "gpt", "value": entry["spanish"]}
    ]
    converted_dataset.append(conversation)
from datasets import Dataset
dataset=Dataset.from_list(converted_dataset[:100])


In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = False,
)



In [ ]:
f = model.base_model.forward


In [ ]:
from unsloth.chat_templates import get_chat_template
def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }

In [ ]:
from unsloth.chat_templates import standardize_sharegpt
dataset = standardize_sharegpt(dataset)
dataset = dataset.map(formatting_prompts_func, batched = True,)

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

In [ ]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|im_start|>user\n",
    response_part = "<|im_start|>assistant\n",
)

In [ ]:
trainer_stats = trainer.train()


In [ ]:
import torch
print(torch.__version__)

In [ ]:
import json
with open("/home/mshahidul/project1/results_new/Medline/medlineplus_gpt4_mini_COD_back_translation.json") as f:
    medline_data = json.load(f)

print(medline_data[0].keys())
li=[]
for x in medline_data:
    li.append({
        "sentence":x['Original_English_sentence'],
        'COD_prompt':x['COD_prompt']
    })


dict_keys(['Original_English_sentence', 'Original_Spanish_sentence', 'COD_prompt', 'spanish_translation_prompt', 'spanish_translation_direct', 'back_translation_prompt', 'back_translation_direct', 'scores_cod_prompt(bleu and chrf)', 'scores_direct(bleu and chrf)'])
